In [23]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/media/paulati/Nuevo vol/paula/dh/2021/dsad_2021_paula/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
nltk=3.5 already installed
unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Modelos de Ensamble

## Introducción

Usando el dataset Movie_classification, que vimos en la clase de CART, vamos a entrenar tres modelos base de clasificación, y construir a partir de ellos un ensamble para predecir el valor de la variable Start_Tech_Oscar.

Vamos a comparar la performance de los modelos base con la del modelo de ensamble.


## Dataset

En esta clase usaremos un dataset con info de películas ("Movie_classification.csv").  
Este dataset esta conformado por los siguientes features:  

 *   **Marketing expense:**    (float64)    Gasto total en Marketing      
 *   **Production expense:**   (float64)    Gasto total de Producción
 *   **Multiplex coverage:**   (float64)    Cobertura promedio de Multiplex
 *   **Budget:**               (float64)    Presupuesto
 *   **Movie_length:**         (float64)    Duración de la película
 *   **Lead_ Actor_Rating:**   (float64)    Puntaje sobre el actor principal
 *   **Lead_Actress_rating:**  (float64)    Puntaje sobre la actriz principal
 *   **Director_rating:**      (float64)    Puntaje sobre el Director
 *   **Producer_rating:**      (float64)    Puntaje sobre el Productor
 *   **Critic_rating:**        (float64)    Puntaje que le puso la crítica
 *   **Trailer_views:**        (int64)      Cantidad de vistas del Trailer
 *   **3D_available:**         (object)     Si esta disponible en 3D (Yes/No)
 *   **Time_taken:**           (float64)    Duración de la película
 *   **Twitter_hastags:**      (float64)    Cantidad de menciones en twitter
 *   **Genre:**                (object)     Genero de la película
 *   **Avg_age_actors:**       (int64)      Edad promedio de los actores
 *   **Num_multiplex:**        (int64)      Cantidad de Multiplex
 *   **Collection:**           (int64)      Recaudación
 *   **Start_Tech_Oscar:**     (int64)      Si recibió un oscar o no.
 

## Imports

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from scipy.stats import mode
import seaborn as sns

## Ejercicio 1

Leamos el dataset "Movie_classification", eliminemos los registros con valores faltantes y las features categóricas "Genre" y "3D_available"

In [25]:
data_raw = pd.read_csv("../Data/Movie_classification.csv")
print(data_raw.shape)

(506, 19)


In [26]:
data_no_missing = data_raw.dropna()
print(data_no_missing.shape)

(494, 19)


In [27]:
data = data_no_missing.drop(["Genre", "3D_available"], axis = 1)
print(data.shape)

(494, 17)


In [28]:
data.head()

,Marketing expense,Production expense,Multiplex coverage,Budget,Movie_length,Lead_ Actor_Rating,Lead_Actress_rating,Director_rating,Producer_rating,Critic_rating,Trailer_views,Time_taken,Twitter_hastags,Avg_age_actors,Num_multiplex,Collection,Start_Tech_Oscar
0,20.1264,59.62,0.462,36524.125,138.7,7.825,8.095,7.910,7.995,7.94,527367,109.60,223.840,23,494,48000,1
1,20.5462,69.14,0.531,35668.655,152.4,7.505,7.650,7.440,7.470,7.44,494055,146.64,243.456,42,462,43200,0
2,20.5458,69.14,0.531,39912.675,134.6,7.485,7.570,7.495,7.515,7.44,547051,147.88,2022.400,38,458,69400,1
3,20.6474,59.36,0.542,38873.890,119.3,6.895,7.035,6.920,7.020,8.26,516279,185.36,225.344,45,472,66800,1
4,21.3810,59.36,0.542,39701.585,127.7,6.920,7.070,6.815,7.070,8.26,531448,176.48,225.792,55,395,72400,1


## Ejercicio 2

Crear los conjuntos de train y test

In [29]:
X = data.drop("Start_Tech_Oscar", axis = 1)
print(X.shape)

y = data.Start_Tech_Oscar
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 127)


(494, 16)
(494,)


## Ejercicio 3

Estandarizar las features para los conjuntos de train y test

In [30]:
scaler = StandardScaler()
X_train_scl = scaler.fit_transform(X_train)

X_test_scl = scaler.transform(X_test)

## Ejercicio 4

Entrenar un modelo Naive Bayes para predecir el valor de "Start_Tech_Oscar" y evaluar su perfomance en test mediante

* accuracy

* matriz de confusión
    


In [31]:
model_1 = GaussianNB()
fit_1 = model_1.fit(X_train_scl, y_train)

In [32]:
predict_1_cat = fit_1.predict(X_test_scl)
accuracy_1 = accuracy_score(y_test, predict_1_cat)
print(accuracy_1)
conf_mat_1 = confusion_matrix(y_test, predict_1_cat)
print(conf_mat_1)

0.5887096774193549
[[49  7]
 [44 24]]


## Ejercicio 5

Entrenar un modelo KNN para predecir el valor de "Start_Tech_Oscar" y evaluar su perfomance en test mediante

* accuracy

* matriz de confusión


In [33]:
model_2 = KNeighborsClassifier()
fit_2 = model_2.fit(X_train_scl, y_train)

In [34]:
predict_2_cat = fit_2.predict(X_test_scl)
accuracy_2 = accuracy_score(y_test, predict_2_cat)
print(accuracy_2)
conf_mat_2 = confusion_matrix(y_test, predict_2_cat)
print(conf_mat_2)

0.6370967741935484
[[34 22]
 [23 45]]


## Ejercicio 6

Entrenar un árbol de clasificación para predecir el valor de "Start_Tech_Oscar" y evaluar su perfomance en test mediante

* accuracy

* matriz de confusión


In [35]:
model_3 = tree.DecisionTreeClassifier(criterion='gini')
fit_3 = model_3.fit(X_train_scl, y_train)

In [36]:
predict_3_cat = fit_3.predict(X_test_scl)
accuracy_3 = accuracy_score(y_test, predict_3_cat)
print(accuracy_3)
conf_mat_3 = confusion_matrix(y_test, predict_3_cat)
print(conf_mat_3)

0.6209677419354839
[[39 17]
 [30 38]]


## Ejercicio 7


Construir un modelo de ensamble usando como modelos base los tres modelos anteriores.

Para esto, escribir una función `predict_ensamble`, que calcule el valor de la etiqueta Start_Tech_Oscar como la moda de las respuestas de los predictores base

 Evaluar la performance del ensamble mediante 

* accuracy

* matriz de confusión




In [37]:
def predict_ensamble(X, model_1, model_2, model_3):
    y_pred_1 = model_1.predict(X)
    y_pred_2 = model_2.predict(X)
    y_pred_3 = model_3.predict(X)
    result_mode = mode([y_pred_1, y_pred_2, y_pred_3]).mode
    result = np.transpose(result_mode)    
    return result

In [38]:
predict_cat_ensemble = predict_ensamble(X_test_scl, fit_1, fit_2, fit_3)
accuracy_ensemble = accuracy_score(y_test, predict_cat_ensemble)
print(accuracy_ensemble)
predict_cat_ensemble.shape
conf_mat_ensemble = confusion_matrix(y_test, predict_cat_ensemble)
print(conf_mat_ensemble)

0.6048387096774194
[[41 15]
 [34 34]]


## Ejercicio 8

¿Cómo es la performance del modelo de ensamble respecto de la obtenida en los modelos base?

¿Mejora usando dos modelos en el ensable? ¿Qué modelos usarían?

En este caso, la performance del modelo de ensamble no es mejor que la del mejor de los modelos base.

In [39]:
def predict_ensamble_2(X, model_1, model_2):
    y_pred_1 = model_1.predict(X)
    y_pred_2 = model_2.predict(X)    
    result_mode = mode([y_pred_1, y_pred_2]).mode
    result = np.transpose(result_mode)    
    return result


In [40]:
predict_cat_ensemble_2_3 = predict_ensamble_2(X_test_scl, fit_2, fit_3)
accuracy_ensemble_2_3 = accuracy_score(y_test, predict_cat_ensemble_2_3)
print(accuracy_ensemble_2_3)
accuracy_ensemble_2_3.shape
conf_mat_ensemble_2_3 = confusion_matrix(y_test, predict_cat_ensemble_2_3)
print(conf_mat_ensemble_2_3)

0.6290322580645161
[[47  9]
 [37 31]]


La performance es muy similar al ensamble de tres modelos, peor que la del mejor modelo base.

Probamos las combinaciones 1, 2 y 1, 3 (vemos que no mejora):

In [41]:
predict_cat_ensemble_1_2 = predict_ensamble_2(X_test_scl, fit_1, fit_2)
accuracy_ensemble_1_2 = accuracy_score(y_test, predict_cat_ensemble_1_2)
print(accuracy_ensemble_1_2)
accuracy_ensemble_1_2.shape
conf_mat_ensemble_1_2 = confusion_matrix(y_test, predict_cat_ensemble_1_2)
print(conf_mat_ensemble_1_2)

0.5725806451612904
[[51  5]
 [48 20]]


In [42]:
predict_cat_ensemble_1_3 = predict_ensamble_2(X_test_scl, fit_1, fit_3)
accuracy_ensemble_1_3 = accuracy_score(y_test, predict_cat_ensemble_1_3)
print(accuracy_ensemble_1_3)
accuracy_ensemble_1_3.shape
conf_mat_ensemble_1_3 = confusion_matrix(y_test, predict_cat_ensemble_1_3)
print(conf_mat_ensemble_1_3)

0.5806451612903226
[[53  3]
 [49 19]]
